## SHEL fitting examples

This notebook shows how to use the utilities in the SHEL_project repository to fit transiting exoplanet parameters. Note that the data is stored in the `shel_database.sqlite` repository - I also show a couple examples of how to load priors and posteriors into the database, as well as a couple example queries.

In [ ]:
import sqlite3 as sql
from fitter import SHEL_Fitter
from utils import load_prior, load_results, detect_outlier_results

#### Connect to database

If you want to interact with the database directly rather than through the utility methods, you'll need a connection and cursor, initialized as below. Note that if you aren't in the same directory as the database file, you'll need to provide the full path, or else the `connect` call will create a new file in your current directory.

In [ ]:
conn = sql.connect('shel_database.sqlite')
cur = conn.cursor()

### Loading a new or updated prior

System parameters (e.g., P_p1 t0_p1, ecc_p1) can be loaded into the `system_parameters` table with the `utils.load_prior` method. The inputs are the target name, parameter, prior value, and prior error. Stellar parameters (e.g. rho) aren't handled by this method, as they go into a separate `stellar_parameters` table. 

In [ ]:
# Load a system parameter prior.
load_prior("HAT-P-26", "P_p1", 4.23452, 1.5e-05)

# Get target id
target_id = cur.execute("select id from targets where name = 'HAT-P-26'").fetchone()[0]

# Load stellar parameters
stmt = f'insert into stellar_parameters (target_id, rho, rho_err) values ({target_id}, 2045.7, 110)'

# Or if the target is already in the table, update
#stmt = f'update stellar_parameters set rho=2045.7, rho_err=110 where target_id = {target_id}'

# Execute whichever statement is needed
cur.execute(stmt)

# You have to commit the changes to have them saved
conn.commit()

### Running a fit

This consists of four steps, one of which is optional:
1. Initialize the fitter object
2. Separately fit the TESS systematics (optional)
3. Initialize the fitter. Most of the options are set in this step.
4. Actually run the fit.

By default, the outputs of the fit will be written to `juliet_fits/[target name]/`. In steps 1 and 2 you can specify an `out_folder_suffix`, which will cause the output to be written to `juliet_fits/[target name][out_folder_suffix]`, for example `juliet_fits/WASP-26_no_tess/`.

In [ ]:
# Provide the target name when initializing the fitter
fitter = SHEL_Fitter("HAT-P-26")

In [ ]:
fitter.fit_tess_systematics()

If you did not run the TESS systematics fit separately, you can set `fit_oot=True` in the `initialize_fit` call to fit all the TESS (and other light curve) data. If that parameter is not set, the light curve data will be trimmed to +/- 1.5 transit widths (this was previously smaller, but if the P/t0 priors aren't great it got close to cutting off actual transit data in some cases).

Other options include excluding RV or light curve data from some sources (to avoid duplicate data that is present in multiple references) and setting eccentricity and omega (and their errors). There is also a flag to only fit the TESS data (`TESS_only=True`).

In [ ]:
fitter.initialize_fit(exclude_rv_sources=[42,])

In [ ]:
# Finally we call this to actually kick off the fit. This uses the dynamic_dynesty sampler.
fitter.run_fit()

### Load posteriors and check for outliers

Note that the `load_results` function requires you to go into the posteriors.dat file and replace any tabs with space before running it. 

In [ ]:
load_results('HAT-P-26')

This outlier detection will only check results that have a last_update timestamp. You can specify a minimum timestamp with `last_update_min = [unix timestamp]`. This returns target/parameter combinations where the posterior 3-sigma range and prior 3-sigma range do not overlap.

In [ ]:
detect_outlier_results()